In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
##########################################
# show pointclouds
from open3d import *
#  K = (0.89115971  0           0.5)
#      (0           1.18821287  0.5)
#      (0           0           1  ),


In [ ]:
import time
import cv2
import pickle
import numpy as np
from numpy.linalg import inv

import torch
import torch.backends.cudnn as cudnn
from torch import Tensor

from depthNet_model import depthNet
from visualize import *


# model
depthnet = depthNet()
model_data = torch.load('opensource_model.pth.tar')
depthnet.load_state_dict(model_data['state_dict'])
depthnet = depthnet.cuda()
cudnn.benchmark = True
depthnet.eval()
# for warp the image to construct the cost volume
pixel_coordinate = np.indices([320, 256]).astype(np.float32)
pixel_coordinate = np.concatenate(
    (pixel_coordinate, np.ones([1, 320, 256])), axis=0)
pixel_coordinate = np.reshape(pixel_coordinate, [3, -1])

In [ ]:
from __future__ import print_function, division
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
# imageio.plugins.freeimage.download()
# from imageio.plugins import freeimage
import json
import cv2
import sys
sys.path.insert(-1, '../DeepNotebook/utils/')
from transformations import *

In [ ]:
ROOT = '/home/komatsu/datasets/demon/traindata/'
# ROOT = '/home/komatsu/datasets/demon/testdata/'

# dataset = 'mvs_achteck_turm'
# dataset = 'mvs_breisach'
# dataset = 'mvs_citywall'
dataset = 'sun3d_train_0.01m_to_0.1m'
# dataset = 'sun3d_train_0.2m_to_0.4m'
# dataset = 'rgbd_10_to_20_3d_train'
# dataset = 'rgbd_10_to_20_simple_train'
# dataset = 'sun3d_test'

with open(os.path.join(ROOT, dataset, "num_images.json")) as f:
    num_images = json.load(f)
print(len(num_images))

In [ ]:
def getFrame(scene, idx):
    fname = os.path.join(ROOT,dataset,'{:04d}/images/{:04}.png'.format(scene, idx))
    img = imageio.imread(fname)
    fname = os.path.join(ROOT,dataset,'{:04d}/depths/{:04}.exr'.format(scene, idx))
    depth = imageio.imread(fname)
    fname = os.path.join(ROOT,dataset,'{:04d}/poses/{:04}.json'.format(scene, idx))
    with open(fname) as f:
        cam = json.load(f)
    return img, depth, cam

def getPCD(img, depth, cam):
    height, width, _ = img.shape
    cam_o3 = PinholeCameraIntrinsic(width, height, cam['f_x'], cam['f_y'], cam['c_x'], cam['c_y'])
    rgbd_image = create_rgbd_image_from_color_and_depth(
        Image(img), Image(depth), depth_scale=1.0, depth_trunc=100, convert_rgb_to_intensity=False)
    pcd = create_point_cloud_from_rgbd_image(rgbd_image, cam_o3)
    # Flip it, otherwise the pointcloud will be upside down
#     pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    return pcd

In [ ]:
scene = 100
left_image, left_depth, left_cam = getFrame(scene, 0)
right_image, right_depth, right_cam = getFrame(scene, 1)
fig, ax = plt.subplots(1,3)
ax[0].imshow(left_image)
ax[1].imshow(right_image)
ax[2].imshow(left_depth)


In [ ]:
pcd = getPCD(left_image, left_depth, left_cam)
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
draw_geometries([pcd])

In [ ]:
left_pose = inv(left_cam['extrinsic'])
right_pose = inv(right_cam['extrinsic'])

fx = left_cam['f_x']
fy = left_cam['f_y']
cx = left_cam['c_x']
cy = left_cam['c_y']
camera_k = np.asarray([
    [fx, 0, cx], 
    [0, fy, cy], 
    [0, 0, 1]])

In [ ]:
right_image = cv2.cvtColor(right_image, cv2.COLOR_RGB2BGR)
left_image = cv2.cvtColor(left_image, cv2.COLOR_RGB2BGR)

# test the epipolar line
left2right = np.dot(inv(right_pose), left_pose)
test_point = np.asarray([left_image.shape[1] // 2, left_image.shape[0] // 2, 1])
far_point = np.dot(inv(camera_k), test_point) * 50.0
far_point = np.append(far_point, 1)
far_point = np.dot(left2right, far_point)
far_pixel = np.dot(camera_k, far_point[0:3])
far_pixel = (far_pixel // far_pixel[2])[0:2]
near_point = np.dot(inv(camera_k), test_point) * 0.1
near_point = np.append(near_point, 1)
near_point = np.dot(left2right, near_point)
near_pixel = np.dot(camera_k, near_point[0:3])
near_pixel = (near_pixel // near_pixel[2])[0:2]
cv2.line(right_image, 
        (int(far_pixel[0] + 0.5), int(far_pixel[1] + 0.5)),
        (int(near_pixel[0] + 0.5), int(near_pixel[1] + 0.5)), [0,0,255], 4)
cv2.circle(left_image,(test_point[0], test_point[1]), 4, [0,0,255], -1)

# cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# scale to 320x256
original_width = left_image.shape[1]
original_height = left_image.shape[0]
factor_x = 320.0 / original_width
factor_y = 256.0 / original_height

left_image = cv2.resize(left_image, (320, 256))
right_image = cv2.resize(right_image, (320, 256))
camera_k[0, :] *= factor_x
camera_k[1, :] *= factor_y

# convert to pythorch format
torch_left_image = np.moveaxis(left_image, -1, 0)
torch_left_image = np.expand_dims(torch_left_image, 0)
torch_left_image = (torch_left_image - 81.0)/ 35.0
torch_right_image = np.moveaxis(right_image, -1, 0)
torch_right_image = np.expand_dims(torch_right_image, 0)
torch_right_image = (torch_right_image - 81.0) / 35.0

# process
left_image_cuda = Tensor(torch_left_image).to(device)
right_image_cuda = Tensor(torch_right_image).to(device)


left_in_right_T = left2right[0:3, 3]
left_in_right_R = left2right[0:3, 0:3]
K = camera_k
K_inverse = inv(K)
KRK_i = K.dot(left_in_right_R.dot(K_inverse))
KRKiUV = KRK_i.dot(pixel_coordinate)
KT = K.dot(left_in_right_T)
KT = np.expand_dims(KT, -1)
KT = np.expand_dims(KT, 0)
KT = KT.astype(np.float32)
KRKiUV = KRKiUV.astype(np.float32)
KRKiUV = np.expand_dims(KRKiUV, 0)
KRKiUV_cuda_T = Tensor(KRKiUV).to(device)
KT_cuda_T = Tensor(KT).to(device)

predict_depths = depthnet(left_image_cuda, right_image_cuda, KRKiUV_cuda_T,
                            KT_cuda_T)

# visualize the results
idepth = np.squeeze(predict_depths[0].cpu().data.numpy())
np_depth = np2Depth(idepth, np.zeros(idepth.shape, dtype=bool))



result_image = np.concatenate(
    (left_image, right_image, np_depth), axis=1)
cv2.imshow("result", result_image)
cv2.waitKey(0)



width, height, _ = left_image.shape
fx = 1.0
fy = 1.0
cx = 0.5
cy = 0.5
cam_o3 = PinholeCameraIntrinsic(width, height, fx*width, fy*height, cx*width, cy*height)
cam_o3.intrinsic_matrix = K

depth = 1/idepth
left = left_image[:,:,::-1].copy()
rgbd_image = create_rgbd_image_from_color_and_depth(
    Image(left), Image(depth), depth_scale=1.0, depth_trunc=30, convert_rgb_to_intensity=False)
pcd = create_point_cloud_from_rgbd_image(rgbd_image, cam_o3)
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
draw_geometries([pcd])


cv2.destroyAllWindows()